In [1]:
[all  true swap [&] step] inscribe
[any false swap [|] step] inscribe

In [2]:
[base 2147483648] inscribe

Model bigints as a pair of Boolean for the sign and a list of integers for the digits, to keep things simple let the bool be the first item on a list followed by zero or more int digits.  The digits shall be confined to the range zero to `pow(2, 31) - 1`

Another way to say that is that our numbers are in base `2147483648` and our "nine" is `2147483647` (`0b1111111111111111111111111111111`, 31 ones.)

This lets us use (Oberon RISC) 32-bit signed ints to store our digits.

In [3]:
[valid_digit [0 >] [base <] &&] inscribe

In [4]:
clear
32 valid_digit 1232147483648 valid_digit

32 true 1232147483648 false

In [5]:
clear
[true 3 2 1]
rest [valid_digit] map all

true

In [6]:
clear
[true 3 2 1]

[true 3 2 1]

Because we are working with Python Joy right now we can convert ints to bigints.

To get the sign bool we can just use `!-` ("not negative")

In [7]:
clear
23 !- -23 !-

true false

To get the list of digits...

```
    @staticmethod
    def digitize(n):
        if n < 0:
            raise ValueError(f'Non-negative only: {n}')
        while n:
            n, digit = divmod(n, 2**31)
            yield OberonInt(digit)

```

    [0 >=] [base divmod swap] while

In [8]:
clear

12345678901234567890 base divmod swap

1797196498 5748904729

In [9]:
base divmod swap

1797196498 1453937433 2

In [10]:
clear

1234567890123456789012345678901234567890
   [0 >] [base divmod swap] while pop

1312754386 1501085485 57659106 105448366 58

In [11]:
clear

[1234567890123456789012345678901234567890]
   [ [0 >] [base divmod swap] while pop ]
   infra

[58 105448366 57659106 1501085485 1312754386]

In [12]:
clear

1234567890123456789012345678901234567890
   [0 <=] [pop []] [base divmod swap] [i cons]

1234567890123456789012345678901234567890 [0 <=] [pop []] [base divmod swap] [i cons]

In [13]:
genrec

[1312754386 1501085485 57659106 105448366 58]

In [14]:
clear

0 [0 <=] [pop []] [base divmod swap] [i cons] genrec

[]

In [15]:
clear

[digitalize [0 <=] [pop []] [base divmod swap] [i cons] genrec] inscribe

In [16]:
clear

1234567890123456789012345678901234567890 digitalize

[1312754386 1501085485 57659106 105448366 58]

So `!-` for the sign and `abs digitalize` for the digits, followed by `cons`:

In [17]:
clear

1234567890123456789012345678901234567890 [!-] [abs digitalize] cleave cons

[true 1312754386 1501085485 57659106 105448366 58]

In [18]:
[to_bigint [!-] [abs digitalize] cleave cons] inscribe

[true 1312754386 1501085485 57659106 105448366 58]

In [19]:
clear

1234567890123456789012345678901234567890 to_bigint

[true 1312754386 1501085485 57659106 105448366 58]

In [20]:
clear

-1234567890123456789012345678901234567890 to_bigint

[false 1312754386 1501085485 57659106 105448366 58]

In [21]:
clear

## Addition of Like Signs `add-digits`

Let's figure out how to add two lists of digits (we will assume that the signs are the same.)  We need to put an inital `false` value for a carry flag, and then there's a `genrec`.

    initial-carry ≡ false rollup
      add-digits' ≡ [P] [THEN] [R0] [R1] genrec
      
       add-digits ≡ initial-carry add-digits'

### The predicate

I think we're going to want a recursive function (duh!?) but it's not quite a standard *hylomorphism* for (at least) two reasons:

- We're tearing down two lists simultaneously.
- They might not be the same length.

There are two base cases: two empty lists or one empty list, the recursive branch is taken only if both lists at non-empty.

    bool [a ...] [b ...] P

The first thing to do is convert them to Booleans:

In [22]:
clear
[
[[a] [b]]
[[a] []]
[[] [b]]
[[] []]
]
[[[bool] ii] infra] 
map

[[true true] [true false] [false true] [false false]]

In [23]:
clear
[
[[a] [b]]
[[a] []]
[[] [b]]
[[] []]
]
[[[bool] ii & not] infra] 
map

[[false] [true] [true] [true]]

    P == [bool] ii & not

### The base cases
We have to decide between three cases, but because addition we can lump together the first two cases:

    bool [] [b ...] THEN
    bool [a ...] [] THEN
    bool [] [] THEN

    THEN ≡ [P'] [THEN'] [ELSE] ifte

In [24]:
clear
[
[[a] []]
[[] [b]]
[[] []]
]
[[[bool] ii |] infra] 
map

[[true] [true] [false]]

    P' ≡ [bool] ii |

So `THEN'` deals with one number (list of digits) being longer than the other, while the `ELSE` branch deals with the case of both lists being empty.

#### One list empty
In the cases where one of the two lists (but not both) is empty:

    carry [a ...] [] THEN'
    carry [] [b ...] THEN'

We first get rid of the empty list:

    ditch-empty-list ≡ [bool] [popd] [pop] ifte

In [25]:
clear
[ditch-empty-list [bool] [popd] [pop] ifte] inscribe

In [26]:
[1][] ditch-empty-list

[1]

In [27]:
clear
[][1] ditch-empty-list

[1]

    THEN' ≡ ditch-empty-list THEN''

Now we have:

    carry [n ...] THEN''

This is `add-carry-to-digits` that kicked my ass earlier today.  Like I mentioned above, I think it was because I put it in the recursive branch!  D'oh!

#### But first  `add-with-carry`
We will want some function `F` that accepts a bool and two ints and leaves behind a new int and a new Boolean carry flag:

            carry0 a b F
    --------------------------
         (a+b+carry0) carry

(I find it interesting that this function accepts the carry from below the int args but returns it above the result.  Hmm...)

In [28]:
clear

[bool_to_int [0] [1] branch] inscribe

false bool_to_int
true bool_to_int

0 1

In [29]:
clear

false 1 2 [bool_to_int] dipd + +

3

In [30]:
clear

true 1 2 [bool_to_int] dipd + +

4

So the first part of `F` is `[bool_to_int] dipd + +` to get the total, then we need to

In [31]:
clear

4 base [mod] [>=] clop

4 false

In [32]:
clear

base 100 +

base [mod] [>=] clop

100 true

Put it all together and we have:

    _add-with-carry0 ≡ [bool_to_int] dipd + +
    _add-with-carry1 ≡ base [mod] [>=] clop

    add-with-carry ≡ _add-with-carry0 _add-with-carry1


In [33]:
clear
[_add-with-carry0 [bool_to_int] dipd + +] inscribe
[_add-with-carry1 base [mod] [>=] clop] inscribe
[add-with-carry _add-with-carry0 _add-with-carry1] inscribe

In [34]:
clear

false base 100 add-with-carry

100 true

In [35]:
clear

true base 100 add-with-carry

101 true

In [36]:
clear

false 2 100 add-with-carry

102 false

In [37]:
clear

true 2 100 add-with-carry

103 false

In [38]:
clear

### Now back to `add-carry-to-digits`

This should be a very simple recursive function.

    add-carry-to-digits ≡ [_actd_P] [_actd_THEN] [_actd_R0] [_actd_R1] genrec

    carry [n ...] add-carry-to-digits
    carry [n ...] [_actd_P] [_actd_THEN] [_actd_R0] [_actd_R1] genrec

The predicate is the carry flag itself inverted, but when we recur we need to check if the list is non-empty because it will eventually be empty or the carry flag will be false:

    _actd_P ≡ pop not

The base case simply discards the carry flag:

    _actd_THEN ≡ popd

That leaves the recursive branch:

    true [n ...] R0 [add-carry-to-digits] R1

-or-

    true [] R0 [add-carry-to-digits] R1

We know that the Boolean value is `true`.  We also know that the list will be non-empty, but only on the first iteration of the `genrec`.  It may be that the list is empty on a later iteration.

The `R0` function should check the list.

    _actd_R0 ≡ [bool] [_actd_R0.then] [_actd_R0.else] ifte

If it's empty...

       true [] R0.else [add-carry-to-digits] R1
    ----------------------------------------------
           false [1] [add-carry-to-digits] R1

    R0.else ≡ 1 swons popd false swap

If it's not empty...

       true [a ...] R0.then [add-carry-to-digits] R1
    -------------------------------------------------------------
       true 0 a add-with-carry [...1] [add-carry-to-digits] R1
    -------------------------------------------------------------
       (a+1) carry [...1] [add-carry-to-digits] R1

Ah ha!?  How to get that `(a+1)` back into the list!?

If it's empty...

       true [] R0.else [add-carry-to-digits] R1
    ------------------------------------------------
          1 false [] [add-carry-to-digits] i cons

and

       true [a ...] R0.then [add-carry-to-digits] i cons
    -------------------------------------------------------------
       true 0 a add-with-carry [...1] [add-carry-to-digits] i cons
    -------------------------------------------------------------
       (a+1) carry [...1] [add-carry-to-digits] i cons

There we go.

    _actd_R0.else ≡ popd 1 false rolldown

    _actd_R0.then ≡ 0 swap uncons [add-with-carry] dip

    _actd_R1 ≡ i cons

In [39]:
[add-carry-to-digits [_actd_P] [_actd_THEN] [_actd_R0] [_actd_R1] genrec] inscribe
[_actd_P pop not] inscribe
[_actd_THEN popd] inscribe
[_actd_R0 [bool] [_actd_R0.then] [_actd_R0.else] ifte] inscribe
[_actd_R0.else popd 1 false rolldown] inscribe
[_actd_R0.then 0 swap uncons [add-with-carry] dip] inscribe
[_actd_R1 i cons] inscribe

In [40]:
clear

false [3 2 1] add-carry-to-digits

[3 2 1]

In [41]:
clear

true [] add-carry-to-digits

[1]

In [42]:
clear

true [3 2 1] add-carry-to-digits

[4 2 1]

In [43]:
clear

true base -- [2 1] cons 

true [2147483647 2 1]

In [44]:
add-carry-to-digits

[0 3 1]

So that handles the cases where one of the two lists (but not both) is empty.

#### Both lists empty
If both lists are empty we discard one list and check the carry to determine our result as decribed above:

    ELSE ≡ pop swap [] [1 swons] branch

In [45]:
clear

true [] [] pop swap [] [1 swons] branch

[1]

In [46]:
clear

false [] [] pop swap [] [1 swons] branch

[]

In [47]:
clear

The story so far...

    add-digits ≡ initial-carry add-digits'
    
    add-digits' ≡ [P] [THEN] [R0] [R1] genrec

    initial-carry ≡ false rollup

    P ≡ [bool] ii & not

    THEN ≡ [P'] [THEN'] [ELSE] ifte

    P' ≡ [bool] ii |

    THEN' ≡ ditch-empty-list add-carry-to-digits

    ELSE ≡ pop swap [] [1 swons] branch

We just need to knock out those recursive functions `R0` and `R1` and we're done.

### And recur...

    bool [a ...] [b ...] R0 [add-digits'] R1

First we will want to `uncons` the digits

In [48]:
clear

false [1 2 3] [4 5 6] [uncons] ii swapd

false 1 4 [2 3] [5 6]

#### `uncons-two`
We could call this `uncons-two`:

In [49]:
clear

[uncons-two [uncons] ii swapd] inscribe

[1 2 3] [4 5 6] uncons-two

1 4 [2 3] [5 6]

In [50]:
clear

    bool a b [...] [...] R0' [add-digits'] R1

It's at this point that we'll want to employ the `add-with-carry` function:

    bool a b [...] [...] [add-with-carry] dipd R0'' [add-digits'] R1

    bool a b add-with-carry [...] [...] R0'' [add-digits'] R1

    (a+b) bool [...] [...] R0'' [add-digits'] R1

If we postulate a `cons` in our `R1` function...

    (a+b) bool [...] [...] R0'' [add-digits'] i cons

Then it seems like we're done?  `R0''` is nothing?

    R0 ≡ uncons-two [add-with-carry] dipd
    
    R1 ≡ i cons


In [51]:
[add-digits initial-carry add-digits'] inscribe
[add-digits' [P] [THEN] [R0] [R1] genrec] inscribe
[initial-carry false rollup] inscribe
[P [bool] ii & not] inscribe
[THEN [P'] [THEN'] [ELSE] ifte] inscribe
[P' [bool] ii |] inscribe
[THEN' ditch-empty-list add-carry-to-digits] inscribe
[ELSE pop swap [] [1 swons] branch] inscribe
[R0 uncons-two [add-with-carry] dipd] inscribe
[R1 i cons] inscribe

In [52]:
[3 2 1] [1 1 1]

[3 2 1] [1 1 1]

In [53]:
add-digits

[4 3 2]

In [54]:
base -- unit

[4 3 2] [2147483647]

In [55]:
add-digits

[3 4 2]

In [56]:
base -- dup dup unit ccons

[3 4 2] [2147483647 2147483647 2147483647]

In [57]:
add-digits

[2 4 2 1]

243 + 999 = 

In [58]:
clear 243 999 +

1242

Bitchin'